In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *

# Covid CV classifier!

Upload a CV scan of covid-19, with a CSV file, using PSTrace - export to CSV. You can upload a single cycle of the scan, or a complete sample. Because of high variance between batches, you can choose different models for different batches.

----

In [2]:
def my_label(): return

In [3]:
path = Path()

In [4]:
models = path.ls(file_exts='.pkl')

In [5]:
learn_inf = load_learner(path/models[0], cpu=True)

In [6]:
model_dropdown = widgets.Dropdown(options=[(str(x)[:-4],x) for x in models])
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [7]:
image_paths = []
def create_image(data):
    global image_paths
    content = "".join([x for x in data.decode('utf-8', errors='ignore') if x in string.printable])
    df = pd.read_csv(io.StringIO(content), error_bad_lines=False, skiprows=[0,1,2,3,4,5,6,7,8],index_col=False)
    
    return_list =[]
    for i in range(len(df.columns))[::2]:
        columns = array(df.iloc[:,[i,i+1]])
        plt.plot(columns[:,0],columns[:,1])
        
        local_path = path/f'{i}.jpg'
        plt.savefig(local_path)
        plt.close()
        
        image_paths.append(local_path)
        return_list.append(PILImage.create(local_path))
    return return_list

In [8]:
def on_model_change(change):
    global learn_inf
    learn_inf = load_learner(path/model_dropdown.value, cpu=True)

In [9]:
def on_data_change(change):
    preds = []
    probs = []
    
    img = create_image(btn_upload.data[-1])
    out_pl.clear_output()
    
    fig=plt.figure()
    axes = []
    for i in range(len(img)): 
        pred,pred_idx,prob = learn_inf.predict(img[i])
        preds.append(pred)
        probs.append(prob[pred_idx])
        
        axes.append( fig.add_subplot(1, len(img), i+1) )
        subplot_title=(f"cycle-{i}\nPrediction: {pred}\nProbability: {prob[pred_idx]:.04f}")
        axes[-1].set_title(subplot_title)  
        plt.imshow(img[i])
        plt.axis('off')
        
    fig.tight_layout()
    out_pl.clear_output()
    
    with out_pl: display(plt.show())
    
    dic = {'Pos':1,'Neg':0}
    my_sum =0
    for i in range(len(preds)):
        if dic[preds[i]]: my_sum  += probs[i] #math.exp(i * -1 / 4) * probs[i]
        else: my_sum  += -1 * probs[i] #math.exp(i * -1 / 4) * -1 * probs[i]
            
    if my_sum > 0:
        pred = 'Pos'
    else: pred = 'Neg'
    
    prob_sum=0
    for i in range(len(preds)):
        if dic[preds[i]]: prob_sum  += probs[i]
        else: prob_sum  += -1 * probs[i]
    prob =abs(prob_sum)/len(probs)
    
    lbl_pred.value = f"Analyzed prediction for the sample: Prediction: {pred}; Probability: {prob}"
    
    global image_paths
    for file in image_paths:
        os.remove(file)
        
    image_paths = []

In [10]:
model_dropdown.observe(on_model_change, 'value')
btn_upload.observe(on_data_change, names=['data'])

In [11]:
display(VBox([HBox([widgets.Label('Choose the model you want to use: '), model_dropdown]),
              widgets.Label('Select your scan - use CSV file format: '), btn_upload, out_pl, lbl_pred]))